# LABELING SENTIMENT WITH LLAMA 3 (OLLAMA)
**NAMA:** GERINDRA JOHN  
**NPM:** 20241310055

Script ini menggunakan Large Language Model (Llama 3) via Ollama untuk melabeli sentimen komentar secara otomatis.

**PENTING UNTUK GOOGLE COLAB:**
Karena Ollama biasanya jalan di Localhost komputer Anda, script ini **TIDAK BISA** langsung jalan di Google Colab kecuali Anda menggunakan teknik *Tunneling* (ngrok) atau menginstall Ollama di dalam Colab.

**Rekomendasi:** Jalankan Notebook ini di **VS Code (Local)**.

In [ ]:
# Install library
!pip install pandas requests tqdm

In [ ]:
import pandas as pd
import requests
import json
from tqdm import tqdm

def npm_20241310055_classify_with_ollama(text, model="llama3"):
    """
    Mengirim prompt ke Ollama Local untuk klasifikasi sentimen.
    """
    # Jika di Colab, URL ini harus diganti dengan URL Ngrok/Public IP komputer Anda
    # Jika di Local VS Code, biarkan localhost
    url = "http://localhost:11434/api/generate"
    
    prompt = f"""
    Analisis sentimen dari komentar berikut tentang Debat Capres Indonesia 2024.
    
    Konteks:
    - POSITIF: Mendukung salah satu paslon (01 Anies/02 Prabowo/03 Ganjar), memuji penampilan debat, atau optimisme Indonesia.
    - NEGATIF: Mengkritik/menghina salah satu paslon, kecewa dengan debat, hate speech, atau keluhan.
    - NETRAL: Informasi datar, pertanyaan tanpa emosi, atau komentar tidak relevan (spam).

    Komentar: "{text}"
    
    Hanya berikan jawaban satu kata: "POSITIF", "NEGATIF", atau "NETRAL".
    Jangan berikan penjelasan apapun.
    Jawaban:
    """
    
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": 0.0
        }
    }
    
    try:
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            result = response.json()['response'].strip().upper()
            result = result.replace('.', '').replace('*', '')
            
            if 'POSITIF' in result: return 'positif'
            if 'NEGATIF' in result: return 'negatif'
            if 'NETRAL' in result: return 'netral'
            return 'netral'
        else:
            print(f"Error Ollama: {response.status_code}")
            return 'netral'
    except Exception as e:
        print(f"Connection Error: {e}")
        return 'error'

def npm_20241310055_run_labeling():
    print("=== MEMULAI LABELING DENGAN LLAMA 3 (NPM: 20241310055) ===")
    
    input_file = 'dataset_youtube_unlabeled.csv'
    output_file = 'dataset_labeled_final.csv'
    
    try:
        df = pd.read_csv(input_file)
        # Limit row jika mau testing dulu di notebook
        # df = df.head(10) 
        print(f"Data dimuat: {len(df)} baris.")
    except FileNotFoundError:
        print(f"[ERROR] File '{input_file}' tidak ditemukan.")
        return

    # Check Connection
    try:
        requests.get("http://localhost:11434")
        print("Koneksi ke Ollama Berhasil!")
    except:
        print("[WARNING] Tidak bisa connect ke Ollama (localhost:11434).")
        print("Jika ini di Colab, pastikan Anda menggunakan tunneling (ngrok).")
        return

    results = []
    for text in tqdm(df['text']):
        label = npm_20241310055_classify_with_ollama(text)
        results.append(label)
        
    df['label'] = results
    df = df[df['label'] != 'error']
    
    df.to_csv(output_file, index=False)
    print(f"\n=== SELESAI ===")
    print(f"Hasil disimpan ke: {output_file}")
    print(df['label'].value_counts())

# Jalankan Fungsi
if __name__ == "__main__":
    npm_20241310055_run_labeling()